<a href="https://colab.research.google.com/github/mbrudd/LLMs/blob/main/Attention%20mechanisms%2C%20part%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tiktoken

In [2]:
raw_text = "This is my dog Hazel"

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
enc_text = tokenizer.encode( raw_text )
print(enc_text)

[1212, 318, 616, 3290, 42805]


In [5]:
vocab_size = 4
output_dim = 8
inputs = torch.nn.Embedding( vocab_size, output_dim )
print( inputs.weight )

Parameter containing:
tensor([[-0.1704, -1.4671,  0.4426, -1.6024, -0.3176,  0.2850,  0.1077,  1.0498],
        [-0.5343,  0.5473,  1.6653,  0.9196, -1.3672,  0.3858,  0.0546, -0.1150],
        [ 0.2694, -0.6523,  0.4049,  0.3974, -0.5747, -0.5147, -1.6517, -0.2605],
        [-0.5086, -0.0952,  0.6344, -1.0397,  1.4200, -0.1959, -0.9147,  1.5092]],
       requires_grad=True)


In [6]:
inputs = inputs.weight.data
inputs

tensor([[-0.1704, -1.4671,  0.4426, -1.6024, -0.3176,  0.2850,  0.1077,  1.0498],
        [-0.5343,  0.5473,  1.6653,  0.9196, -1.3672,  0.3858,  0.0546, -0.1150],
        [ 0.2694, -0.6523,  0.4049,  0.3974, -0.5747, -0.5147, -1.6517, -0.2605],
        [-0.5086, -0.0952,  0.6344, -1.0397,  1.4200, -0.1959, -0.9147,  1.5092]])

In [7]:
inputs.shape

torch.Size([4, 8])

In [8]:
for row in inputs:
    print(row.tolist())

[-0.17037160694599152, -1.4670987129211426, 0.44257688522338867, -1.6024442911148071, -0.31758782267570496, 0.2850172817707062, 0.10768774151802063, 1.0497959852218628]
[-0.5343101620674133, 0.547331690788269, 1.6653246879577637, 0.9196138978004456, -1.367247462272644, 0.38575413823127747, 0.054583583027124405, -0.11504777520895004]
[0.26936084032058716, -0.6523193120956421, 0.40489014983177185, 0.3973643183708191, -0.5746732950210571, -0.5147222280502319, -1.6516999006271362, -0.26049181818962097]
[-0.5086283087730408, -0.09520038217306137, 0.6343787908554077, -1.03965425491333, 1.4199848175048828, -0.1959317922592163, -0.9146639704704285, 1.509229302406311]


In [9]:
x = torch.Tensor([1.1, 2.3])
y = torch.Tensor([3.4,-2.1])


In [10]:
1.1*3.4 + 2.3*(-2.1)

-1.0899999999999999

In [11]:
torch.dot( x, y)

tensor(-1.0900)

In [12]:
query = inputs[2]
print(query)

tensor([ 0.2694, -0.6523,  0.4049,  0.3974, -0.5747, -0.5147, -1.6517, -0.2605])


In [13]:
for i in range( len( inputs )):
    print( torch.dot( query, inputs[i] ) )

tensor(0.0380)
tensor(1.0657)
tensor(4.2111)
tensor(0.1713)


In [14]:
attention_scores_2 = torch.zeros( len(inputs) )
for i in range( len( inputs )):
    attention_scores_2[i] = torch.dot( query, inputs[i] )
print( attention_scores_2)

tensor([0.0380, 1.0657, 4.2111, 0.1713])


In [15]:
# normalize the attention scores using the softmax function:
# def softmax(x):
#     torch.exp(x) / torch.exp(x).sum()

In [16]:
attention_weights_2 = torch.softmax( attention_scores_2, dim = 0 )
attention_weights_2

tensor([0.0143, 0.0400, 0.9293, 0.0164])

In [17]:
attention_weights_2.sum()

tensor(1.)

In [18]:
context_vector_2 = torch.zeros( query.shape )
for i in range( len( attention_weights_2 ) ):
    context_vector_2 += attention_weights_2[i]*inputs[i]
context_vector_2

tensor([ 0.2182, -0.6069,  0.4596,  0.3661, -0.5701, -0.4620, -1.5462, -0.2070])

In [19]:
inputs

tensor([[-0.1704, -1.4671,  0.4426, -1.6024, -0.3176,  0.2850,  0.1077,  1.0498],
        [-0.5343,  0.5473,  1.6653,  0.9196, -1.3672,  0.3858,  0.0546, -0.1150],
        [ 0.2694, -0.6523,  0.4049,  0.3974, -0.5747, -0.5147, -1.6517, -0.2605],
        [-0.5086, -0.0952,  0.6344, -1.0397,  1.4200, -0.1959, -0.9147,  1.5092]])

In [20]:
inputs.T

tensor([[-0.1704, -0.5343,  0.2694, -0.5086],
        [-1.4671,  0.5473, -0.6523, -0.0952],
        [ 0.4426,  1.6653,  0.4049,  0.6344],
        [-1.6024,  0.9196,  0.3974, -1.0397],
        [-0.3176, -1.3672, -0.5747,  1.4200],
        [ 0.2850,  0.3858, -0.5147, -0.1959],
        [ 0.1077,  0.0546, -1.6517, -0.9147],
        [ 1.0498, -0.1150, -0.2605,  1.5092]])

In [21]:
# get all of the attention scores via a matrix multiplication:
attention_scores = inputs @ inputs.T
attention_scores

tensor([[ 6.2409, -1.0193,  0.0380,  3.1521],
        [-1.0193,  6.2384,  1.0657, -1.9206],
        [ 0.0380,  1.0657,  4.2111,  0.1713],
        [ 3.1521, -1.9206,  0.1713,  6.9202]])

In [22]:
attention_weights = torch.softmax( attention_scores, dim = -1 )
attention_weights

tensor([[9.5394e-01, 6.7062e-04, 1.9305e-03, 4.3461e-02],
        [7.0005e-04, 9.9338e-01, 5.6316e-03, 2.8425e-04],
        [1.4317e-02, 4.0009e-02, 9.2932e-01, 1.6357e-02],
        [2.2546e-02, 1.4126e-04, 1.1442e-03, 9.7617e-01]])

In [23]:
attention_weights[0].sum()

tensor(1.)

In [24]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[-0.1845, -1.4046,  0.4517, -1.5724, -0.2433,  0.2626,  0.0598,  1.0665],
        [-0.5295,  0.5390,  1.6571,  0.9144, -1.3613,  0.3804,  0.0447, -0.1146],
        [ 0.2182, -0.6069,  0.4596,  0.3661, -0.5701, -0.4620, -1.5462, -0.2070],
        [-0.5001, -0.1267,  0.6299, -1.0504,  1.3781, -0.1854, -0.8923,  1.4966]])